In [2]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
import glob
import os

In [5]:
def calibrate(showPics = True):
    root = os.getcwd()
    calibrationDir = os.path.join(root, './StereoLeft/')
    imgPathList = glob.glob(os.path.join(calibrationDir, "*.png"))

    nRows = 9
    nCols = 6

    termCrteria = (cv.TERM_CRITERIA_EPS + cv.TermCriteria_MAX_ITER, 30, 0.001)
    worldPtsCur = np.zeros((nRows*nCols, 3), np.float32)
    worldPtsCur[:, :2] = np.mgrid[0:nRows, 0:nCols].T.reshape(-1,2)
    worldPtsList = []
    imgPtsList = []
    i = 0
    for curImgPath in imgPathList:
        i += 1
        imgBGR = cv.imread(curImgPath)
        imgGray = cv.cvtColor(imgBGR, cv.COLOR_BGR2GRAY)
        cornersFound, cornersOrg = cv.findChessboardCorners(imgGray, (nRows, nCols), None)

        if cornersFound == True:
            worldPtsList.append(worldPtsCur)
            cornersRefined = cv.cornerSubPix(imgGray, cornersOrg, (11,11), (-1,-1), termCrteria)
            imgPtsList.append(cornersRefined)
            if showPics:
                cv.drawChessboardCorners(imgBGR, (nRows, nCols), cornersRefined, cornersFound)
                cv.imshow("chessboard", imgBGR)
                cv.imwrite("Image"+str(i)+".png", imgBGR)
                cv.waitKey(0)
    cv.destroyAllWindows()

    repError, camMatrix, distCoeff, rvecs, tvecs = cv.calibrateCamera(worldPtsList, imgPtsList, imgGray.shape[::-1], None, None)

    print("Camera Matrix : \n", camMatrix)
    
    print("Distortion Matirx : \n", distCoeff)
    print("Report Error (pixels) : {:.4f}".format(repError))
    curFolder = os.path.dirname(os.path.abspath("Images"))
    paramPath = os.path.join(curFolder, 'calibration.npz')
    np.savez(paramPath, repError=repError, camMatrix=camMatrix, distCoeff=distCoeff, rvecs=rvecs, tvecs=tvecs)

    return camMatrix, distCoeff

calibrate()

Camera Matrix : 
 [[695.94690533   0.         384.18034585]
 [  0.         694.224292   318.21984931]
 [  0.           0.           1.        ]]
Distortion Matirx : 
 [[ 3.05070760e-03 -4.92009457e-01  2.12421690e-03  1.03882494e-03
   1.82300418e+00]]
Report Error (pixels) : 0.1396


(array([[695.94690533,   0.        , 384.18034585],
        [  0.        , 694.224292  , 318.21984931],
        [  0.        ,   0.        ,   1.        ]]),
 array([[ 3.05070760e-03, -4.92009457e-01,  2.12421690e-03,
          1.03882494e-03,  1.82300418e+00]]))

In [6]:
CHECKERBOARD = (9, 6)

objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

objpoints = []
imgpoints_left = []
imgpoints_right = []

# Load images
images_left = glob.glob('./StereoLeft/*.png')
images_right = glob.glob('./StereoRight/*.png')

# Check if number of images match
assert len(images_left) == len(images_right), "Mismatch between left and right images."

In [7]:
for left_img_path, right_img_path in zip(images_left, images_right):
    left_img = cv.imread(left_img_path)
    right_img = cv.imread(right_img_path)

    gray_left = cv.cvtColor(left_img, cv.COLOR_BGR2GRAY)
    gray_right = cv.cvtColor(right_img, cv.COLOR_BGR2GRAY)

    # Find the checkerboard corners
    ret_left, corners_left = cv.findChessboardCorners(gray_left, CHECKERBOARD, None)
    ret_right, corners_right = cv.findChessboardCorners(gray_right, CHECKERBOARD, None)

    if ret_left and ret_right:
        objpoints.append(objp)
        imgpoints_left.append(corners_left)
        imgpoints_right.append(corners_right)

In [8]:
ret_left, mtx_left, dist_left, rvecs_left, tvecs_left = cv.calibrateCamera(objpoints, imgpoints_left, gray_left.shape[::-1], None, None)
ret_right, mtx_right, dist_right, rvecs_right, tvecs_right = cv.calibrateCamera(objpoints, imgpoints_right, gray_right.shape[::-1], None, None)

flags = 0
flags |= cv.CALIB_FIX_INTRINSIC
retval, mtx_left, dist_left, mtx_right, dist_right, R, T, E, F = cv.stereoCalibrate(
    objpoints, imgpoints_left, imgpoints_right, mtx_left, dist_left, mtx_right, dist_right, gray_left.shape[::-1], flags=flags)

R1, R2, P1, P2, Q, roi_left, roi_right = cv.stereoRectify(
    mtx_left, dist_left, mtx_right, dist_right, gray_left.shape[::-1], R, T)

left_map_x, left_map_y = cv.initUndistortRectifyMap(mtx_left, dist_left, R1, P1, gray_left.shape[::-1], cv.CV_32FC1)
right_map_x, right_map_y = cv.initUndistortRectifyMap(mtx_right, dist_right, R2, P2, gray_right.shape[::-1], cv.CV_32FC1)

np.savez('USB_camera_data.npz',
         left_map_x=left_map_x,
         right_map_x=right_map_x,
         left_map_y=left_map_y,
         right_map_y=right_map_y)

: 